In [1]:
import numpy as np
import pandas as pd
from itertools import product
from scipy.spatial.distance import pdist
from sklearn.model_selection import train_test_split

In [2]:
class KRR():
    def __init__(self, alpha=1.0, gamma=1.0, degree=2, coef0=1, kernel='rbf'):
        self.alpha = alpha  # Ridge parameter.
        self.gamma = gamma  # Gamma parameter for the RBF and polynomial kernels.
        self.degree = degree  # Degree of the polynomial kernel. Ignored by other kernels.
        self.coef0 = coef0  # Zero coefficient for polynomial and sigmoid kernel. Ignored by other kernels.
        self.kernel = kernel  # Kernel mapping.
        self.weights = None
        self.x_train = None
    
    # I will first define all the kernel functions.
    # For many real problems, the underlying model cannot be described
    # by a linear function, so linear ridge regression suffers from poor
    # prediction error. In those cases, a common approach is to map
    # samples to a high dimensional space using a nonlinear mapping,
    # and then learn the model in the high dimensional space. Kernel
    # method is a widely used approach to conduct this learning
    # procedure implicitly by defining the kernel function — the similarity
    # of samples in the high dimensional space.
    # https://scikit-learn.org/stable/modules/metrics.html#metrics
    
    def linear(self, x1, x2):
        """
        A linear kernel function with the formula: Φ(x1, x2) = x1*x2
        """
        linear_kernel = np.dot(x1.T, x2)
        return linear_kernel
    
    def polynomial(self, x1, x2):
        """
        Polynomial kernel function with formula: Φ(x1, x2) = (r + x1 · x2)d, for some r ≥ 0, d > 0
        """
        d = self.degree
        r = self.coef0
        a = self.gamma
        poly_kernel = np.power(r + np.dot(a*x1.T, x2), d)
        return poly_kernel
    
    def rbf(self, x1, x2):
        """
        The radial basis kernel function with formula: Φ(x1, x2) = exp(-a||x1 - x2||2), gamma > 0
        """
        y = self.gamma
        y = y * -1
        rbf_kernel = np.exp(y * np.sum(np.square((x1-x2))))
        return rbf_kernel

    def sigmoid(self, x1, x2):
        """
        The sigmoid kernel function with formula: Φ(x1, x2) = tanh(ax1 * x2 + r )
        """
        a = self.gamma
        r = self.coef0
        sigmoid_kernel = np.tanh(np.dot(a * x1.T, x2) + r)
        return sigmoid_kernel
    
    def kernel_matrix(self, x):
        """
        Create an n*n kernel matrix K using the user-specified kernel function.

        # Use this code to replace the nested loop
        x1, x2 = np.meshgrid(x_train)
        k_mat = kernel_functions[self.kernel](x1, x2)
        """
        # Initialize hash table for kernel functions.
        kernel_functions = {'linear': self.linear, 'rbf': self.rbf, 'poly': self.polynomial, 
                            'sigmoid': self.sigmoid}
        
        # Create an empty array for the matrix. More efficient since we avoid growing our vector.
        k_mat = np.empty((x.shape[0], x.shape[0]), dtype=float)

        # Calculate values for the matrix using the kernel function specified by the user
        for i in range(x.shape[0]):
            for j in range(x.shape[0]):
                km = kernel_functions[self.kernel](x[i], x[j])
                k_mat[i][j] = km
        
        return k_mat

    def fit(self, x, y):
        """
        In the Training phase, the algorithm's goal is to get α by (approximately) solving the linear system;
        (K + λnI)α = y
        α = inverse(K + λnI) * y
        """
        self.x_train = x
        n = x.shape[0]
        # alpha_param λnI
        alpha_param = self.alpha*np.identity(n, dtype=float)
        
        # Construct your kernel matrix
        k_matrix = self.kernel_matrix(x)
        
        # solve for α = inverse(K + λnI) * y
        self.weights = np.dot(np.linalg.inv(np.add(k_matrix, alpha_param)), y)
        
        return self

    def predict(self, x):
        """
        yj = sum(αi*K(xi, xj) where i = 1,...,n
        """
        kernel_functions = {'linear': self.linear, 'rbf': self.rbf, 'poly': self.polynomial, 
                            'sigmoid': self.sigmoid}

        if self.weights is None:
            return "Model not yet trained."

        # Create an empty array for the predicted values to avoid a growing vector/array
        y_pred = np.empty((x.shape[0]), dtype=float)
        # Calculate predicted values
        for j in range(x.shape[0]):
            y_pred[j] = np.sum([np.dot(self.weights[i], kernel_functions[self.kernel](self.x_train[i], x[j])) for i in range(0, self.x_train.shape[0])])
        
        return y_pred
    
    def score(self, x, y):
        y_pred = self.predict(x)
        rmse_score = np.sqrt(np.mean(np.square((y_pred - y))))
        return rmse_score

In [3]:
class GridSearchCV():
    def __init__(self, param_grid, n_splits=5, shuffle=True):
        # Because of time constraint, I only used rmse scoring
        
        self.param_grid = param_grid  # A dictionary
        self.n_splits = n_splits  # Number of folds. Must be atleast 2. Default is 5.
        self.shuffle = shuffle
    
    def grid_parameters(self):
        """
        For constructing a grid of the parameters provided by the user
        param_grid={"alpha": alpha, "gamma": gamma
        For example: grid_parameters({"alpha": [1, 2, 3], "gamma": [4, 5]}) would return: 
        [{'alpha': 1, 'gamma': 4},
        {'alpha': 1, 'gamma': 5},
        {'alpha': 2, 'gamma': 4},
        {'alpha': 2, 'gamma': 5},
        {'alpha': 3, 'gamma': 4},
        {'alpha': 3, 'gamma': 5}]
        https://github.com/scikit-learn/scikit-learn/blob/37ac6788c9504ee409b75e5e24ff7d86c90c2ffb/sklearn/model_selection/_search.py#L50
        """
        grid = []
        items = sorted(self.param_grid.items())  # sort keys of dictionary for reproducibility
        keys, values = zip(*items)
        for v in product(*values):
            params = dict(zip(keys, v))
            grid.append(params)
        return grid
    
    def train(self, x, y):
        """
        This method runs fit with all sets of parameters
        param x: the arrays representing the features
        param y: the arrays representing the labels
        """
        x_folds = np.array_split(x, self.n_splits)
        y_folds = np.array_split(y, self.n_splits)
        scores = {'alpha':[], 'gamma': [], 'rmse': []}
        grids = self.grid_parameters()
        i = 0

        print("Fitting 5 folds for each {} candidates.". format(len(grids)))
        for grid in grids:
            for k in range(self.n_splits):
                # Split the data
                x_train = list(x_folds)
                x_test = x_train.pop(k)
                x_train = np.concatenate(x_train)

                y_train = list(y_folds)
                y_test = y_train.pop(k)
                y_train = np.concatenate(y_train)

                model = KRR(alpha=grid['alpha'], gamma=grid['gamma'])
                model.fit(x_train, y_train)
                score = model.score(x_test, y_test)

                scores['alpha'].append(grid['alpha'])
                scores['gamma'].append(grid['gamma'])
                scores['rmse'].append(score)
            i +=1
        
        print("Finished fitting 5 folds for each {} candidates.". format(len(grids)))
        scores_df = pd.DataFrame(scores)
        best_scores = scores_df[scores_df['rmse'] == scores_df['rmse'].min()]
        
        
        # finally fit the model with the best parameters
        best_alpha = best_scores['alpha'].values[0]
        best_gamma = best_scores['gamma'].values[0]
        best_score = best_scores['rmse'].values[0]

        KRR(alpha=best_alpha, gamma=best_gamma).fit(x, y)
        print('Fitted.')
    
        return best_alpha, best_gamma, best_score

    def predict(self, x):
        """
        predict y for x using the best parameters.
        """
        y_pred = KRR().predict(x)

        return y_pred
    
    def best_paramaters(self):
        """
        Returns best output
        """
        best_alpha = self.best_params.alpha
        best_gamma = self.best_params.gamma
        output = 'Best: alpha = {}, gamma = {}'.format(best_alpha, best_gamma)
        return output

In [4]:
# Load the datasets
data = np.load('toluene.npz')
   
E = data['E'] 
R = data['R']

E -= E.min()
D = np.array([1./pdist(r) for r in R])
print(E.shape, R.shape)
print(D.shape)

(22140, 1) (22140, 15, 3)
(22140, 105)


In [5]:
# split datasets for training
X_train, X_test, y_train, y_test = train_test_split(D, E, train_size=1000, random_state=0)

In [6]:
krr = GridSearchCV(param_grid={"alpha": np.logspace(-12, -6, 3), "gamma": np.array([1e-12, 0.004, 1e-6, 0.04])})
best_alpha, best_gamma, best_score = krr.train(X_train, y_train)

Fitting 5 folds for each 12 candidates.
Finished fitting 5 folds for each 12 candidates.
Fitted.


In [16]:
best_score

4.319678411060403

In [17]:
krr_model = KRR(alpha=best_alpha, gamma=best_gamma)
krr_model.fit(X_train, y_train)
y_train_predicted = krr_model.predict(X_train)
y_test_predicted = krr_model.predict(X_test)

In [18]:
mae = lambda X, Y: np.mean(np.absolute((X - Y)))
rmse = lambda X, Y: np.sqrt(np.mean(np.square((X - Y))))

In [19]:
train_mae = mae(y_train_predicted, y_train)
train_rmse = rmse(y_train_predicted, y_train)
test_mae = mae(y_test_predicted, y_test)
test_rmse = rmse(y_test_predicted, y_test)

print('Train: MAE = {:.3f} kcal/mol, RMSE = {:.3f} kcal/mol'. format(train_mae, train_rmse))
print('Tset: MAE = {:.3f} kcal/mol, RMSE = {:.3f} kcal/mol'. format(test_mae, test_rmse))

Train: MAE = 3.943 kcal/mol, RMSE = 4.933 kcal/mol
Tset: MAE = 4.074 kcal/mol, RMSE = 5.112 kcal/mol


In [ ]:
from matplotlib import pyplot as plt
 
plt.scatter(y_train, y_train_predicted, marker='.', color='green', zorder=2)
plt.scatter(y_test, y_test_predicted, marker='.', color='blue', zorder=1)
plt.title('MAE = {:.3f} kcal/mol, RMSE = {:.3f} kcal/mol'.format(test_mae, test_rmse))
#plt.savefig('krr.png', dpi=600)
plt.show()